This is a page for exploring a the daily peat health indicators for a specific region of interest within a site.

In [ ]:
import geoviews as gv
import panel as pn
import param

import utils.cards
import utils.location
import utils.peat_health_indicators.models
import utils.peat_health_indicators.phi
import utils.template

pn.extension(loading_indicator=True, defer_load=True)
pn.extension("mathjax")
gv.extension("bokeh")

In [ ]:
class Page(utils.location.UrlQueryParams):
    # bind to url query parameter
    indicator_id: str = param.String()  # type: ignore

    phi: utils.peat_health_indicators.phi.BasePHI | None = param.ClassSelector(
        class_=utils.peat_health_indicators.phi.BasePHI, allow_None=True, default=None
    )  # type: ignore

    @param.depends("site_id", "indicator_id", watch=True)
    def update_phi(self):
        directory = utils.peat_health_indicators.models.get_phi(self.site_id, self.indicator_id)
        if directory is None:
            self.phi = None
        else:
            self.phi = utils.peat_health_indicators.phi.DailyPHI.from_directory(directory)

    @param.depends("site", "phi", watch=False)
    def main(self) -> pn.Column:
        """content to render in the main area of a template"""
        elements = []

        if self.site is None:
            elements.append(pn.pane.Alert("Site not found", alert_type="danger"))
        elif self.phi is None:
            elements.append(utils.site_indicators.get_phi(self.site_id, self.indicator_id) is None)
            elements.append(pn.pane.Alert("Indicator not found", alert_type="danger"))
        else:
            site_name = self.site.title or self.site.id
            elements.append(
                pn.pane.Markdown(
                    f"""
                    # Site-level peat health indicators

                    On this page you can interactively explore our site-level peat health indicators
                    for the site {site_name}.
                    """
                    +
                    r"""
                    Below you can see the peat extent classification map,
                    over which we have aggregated the multi-year timeseries of 
                    biogeophysical, hydrological, and surface motion variables.

                    These aggregated timeseries have been prepared at daily resolution,
                    and aligned onto a shared time axis.

                    ## Data processing

                    Each variable is processed as follows:

                    1. Optionally, the variable is transformed relative to some optimal value 
                       $$\delta_t = | x_t - x^* |$$
                    2. Calculate a 365 day climatology with an inverse-variance weighted mean
                    3. Calculate standard anomalies $$z_t$$ relative to the climatology

                    ## Variable loadings
                    
                    Finally we combine all variables into a single integrated 
                    peat health indicator timeseries $$\mathrm{PHI}_t$$
                    using a weighted sum, defined by variable loadings $$l_v \in [-1, +1]$$ for each variable $$v$$.

                    $$
                    \mathrm{PHI}\_t = \frac{ \sum\_v l_v z_{v, t} }{ \sum_{v} | l_{v} | }
                    $$

                    The magnitude of a loading $$l_v$$ indicates the weight or importance of a variable.

                    Whilst the sign of a loading $$l_v$$ indicates whether a variable is positively
                    or negatively correlated with peat health. 
                    For example, to express the hypothesis that healthier peatlands are greener,
                    we would assign a positive loading to a greenness variable
                    such as EVI, FAPAR, or LAI.

                    If a variable has been transformed relative to an optimal value,
                    then a negative loading should be applied,
                    to express that large deviations from the optimal value are associated with poor peat health.

                    You can explore the processing of the individual variables below,
                    and adjust the variable loadings in the sidebar.
                    """
                )
            )
            elements.append(pn.pane.Markdown("## Site"))
            elements.append(utils.cards.site(self.site, collapsed=True))
            elements.append(pn.pane.Markdown("## Peat extent classification"))
            elements.append(
                pn.pane.HoloViews(
                    self.phi.map(),
                    width=600,
                    height=500,
                )
            )
            elements.append(pn.pane.Markdown(self.phi.description))
            elements.append(pn.pane.Markdown("## Individual variables"))
            elements.append(self.phi.variable_cards())
            elements.append(pn.pane.Markdown("## Peat health indicator"))
            elements.append(self.phi)

        return pn.Column(*elements)

    @param.depends("phi", watch=False)
    def sidebar(self) -> pn.Column:
        """content to render in the sidebar of a template"""
        elements = []

        if self.phi is not None:
            elements.append(pn.pane.Markdown("## Variable loadings"))
            elements.append(self.phi.loading_sliders())
            elements.append(self.phi.predefined_variable_loading_selector())

        return pn.Column(*elements)

In [ ]:
page = Page()

In [ ]:
# In a notebook environment pn.state.location is not initialized until the first plot has been displayed

pn.state.location.sync(page, {"site_id": "site-id", "indicator_id": "indicator-id"})  # type: ignore

In [ ]:
template = utils.template.get_template(main=page.main, sidebar=page.sidebar)
template.servable();